In [1]:
import os
import pickle
import glob

import pandas as pd 
import numpy as np

in_dir = "/home/jupyter/unity_jointly_rec_and_search/datasets/spotify_mpd/processed/"

In [2]:
# start create graph

import random 
from tqdm import tqdm
import networkx as nx
from collections import defaultdict
import itertools

fns = ["train_aid_to_simpids.pkl", "test_aid_to_simpids.pkl",
      "train_qid_to_relpids.pkl", "test_qid_to_relpids.pkl"]

datas = []
for fn in fns:
    fn = os.path.join(in_dir, fn)
    with open(fn, "rb") as fin:
        datas.append(pickle.load(fin))

train_aid_to_simpids, test_aid_to_simpids,\
train_qid_to_pids, test_qid_to_pids = datas
print("number of aid_to_simpids  train = {:,}, test = {:,}".format(len(train_aid_to_simpids), 
                                                                     len(test_aid_to_simpids)))
print("number of qid_to_pids train = {:,}, test = {:,}".format(len(train_qid_to_pids), 
                                                                      len(test_qid_to_pids)))

assert len( set(train_aid_to_simpids.keys()) &  set(test_aid_to_simpids.keys()) ) == 0
assert len( set(train_qid_to_pids.keys())  & set(test_qid_to_pids.keys())) == 0

SIM_RELATION = "also_contains"
REL_RELATION = "is_relevant_to"

number of aid_to_simpids  train = 391,094, test = 48,869
number of qid_to_pids train = 393,188, test = 49,149


In [15]:
PIDS = []
with open(os.path.join(in_dir, "all_tracks.tsv")) as fin:
    for line in fin:
        PIDS.append(int(line.strip().split("\t")[0]))
print(f"max pids = {max(PIDS)}")
assert len(PIDS) == len(set(PIDS))

def create_triples(hid, pos_tid, miss_hids, duplicate_pairs, eid_to_text, sampler=None):
    if sampler != None:
        assert type(sampler) == dict or type(sampler) == defaultdict, type(sampler)
        if hid not in sampler:
            miss_hids.append(hid)
            return 0
    if eid_to_text[hid] == eid_to_text[pos_tid]:
        duplicate_pairs.append((hid, pos_tid))
        return 0
    
    if sampler != None and len(sampler[hid]) > 10:
        neg_tid = random.sample(sampler[hid], k=1)[0]
        while neg_tid == pos_tid:
            neg_tid = random.sample(sampler[hid], k=1)[0]
    else:
        neg_tid = random.sample(PIDS, k=1)[0]
        while neg_tid == pos_tid:
            neg_tid = random.sample(range(len(PIDS)), k=1)[0]
            
    return (hid, pos_tid, neg_tid)


eid_to_text = {}
with open(os.path.join(in_dir, "all_entities.tsv")) as fin:
    for line in fin:
        eid, text = line.strip().split("\t")
        eid_to_text[int(eid)] = text

run_path = os.path.join(in_dir, "runs/bm25.all.run")
bm25_hid_to_tids = defaultdict(list)
with open(run_path) as fin:
    for line in fin:
        array = line.strip().split(" ")
        assert len(array) == 6
        hid, tid = int(array[0]), int(array[2])
        if hid in train_qid_to_pids or hid in train_aid_to_simpids:
            bm25_hid_to_tids[hid].append(tid)

max pids = 2256247


'\neid_to_text = {}\nwith open(os.path.join(in_dir, "all_entities.tsv")) as fin:\n    for line in fin:\n        eid, text = line.strip().split("\t")\n        eid_to_text[int(eid)] = text\n\nrun_path = os.path.join(in_dir, "runs/bm25.all.run")\nbm25_hid_to_tids = defaultdict(list)\nwith open(run_path) as fin:\n    for line in fin:\n        array = line.strip().split(" ")\n        assert len(array) == 6\n        hid, tid = int(array[0]), int(array[2])\n        if hid in train_qid_to_pids or hid in train_aid_to_simpids:\n            bm25_hid_to_tids[hid].append(tid)\n'

In [18]:
miss_hids = []
duplicate_pairs = []

h2sp_triples = []
q2p_triples = []
for hid, tail_ids in tqdm(train_aid_to_simpids.items(), total=len(train_aid_to_simpids)):
    for pos_tid in tail_ids:
        triple = create_triples(hid, pos_tid, miss_hids, duplicate_pairs, eid_to_text)
        if triple != 0:
            h2sp_triples.append(triple)
print("miss_hids = {:,}, duplicate_pairs = {:,}".format(len(miss_hids), len(duplicate_pairs)))
print("="*75)
for qid, pos_pids in tqdm(train_qid_to_pids.items(), total=len(train_qid_to_pids)):
    for pos_pid in pos_pids:
        triple = create_triples(qid, pos_pid, miss_hids, duplicate_pairs, eid_to_text, sampler=bm25_hid_to_tids)
        if triple != 0:
            q2p_triples.append(triple)
print("miss_hids = {:,}, duplicate_pairs = {:,}".format(len(miss_hids), len(duplicate_pairs)))
print("="*75)

print("number of h2sp, q2p triples = {:,}, {:,}".format(
    len(h2sp_triples), len(q2p_triples)
))

100%|██████████| 391094/391094 [01:25<00:00, 4571.36it/s]


miss_hids = 0, duplicate_pairs = 0


100%|██████████| 393188/393188 [01:36<00:00, 4090.74it/s]


miss_hids = 438,366, duplicate_pairs = 0
number of h2sp, q2p triples = 24,125,410, 25,638,010


In [19]:
import pickle

kgc_dir = in_dir
out_dir = os.path.join(kgc_dir, "unified_train/")
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    
h2sp_triples = random.sample(h2sp_triples, k=600_000)
q2p_triples = random.sample(q2p_triples, k=600_000)

fn_to_tripleNrel = {
    "a2sp.train.tsv": (h2sp_triples, SIM_RELATION),
    "q2p.train.tsv": (q2p_triples, REL_RELATION),   
}

for fn, (triples, relation) in fn_to_tripleNrel.items():
    with open(os.path.join(out_dir, fn), "w") as fout:
        for (hid, pos_tid, neg_tid) in triples:
            fout.write(f"{hid}\t{pos_tid}\t{neg_tid}\t{relation}\n")
            
out_dir = os.path.join(kgc_dir, "unified_test/")
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

# for similar items
fn_to_aids = {
    "anchors.test.sim.tsv": list(test_aid_to_simpids.keys()),
    "anchors.test.sim.small.tsv": random.sample(list(test_aid_to_simpids.keys()), k=10_000),
}
for fn, aids in fn_to_aids.items():
    with open(os.path.join(out_dir, fn), "w") as fout:
        for aid in aids:
            text = eid_to_text[aid]
            fout.write(f"{aid}\t{text}\t{SIM_RELATION}\n")
            
fn_to_arels = {
    "arels.test.sim.tsv": [(aid, pid) for aid, simpids in test_aid_to_simpids.items() for pid in simpids],
}
for fn, arels in fn_to_arels.items():
    with open(os.path.join(out_dir, fn), "w") as fout:
        for (aid, pid) in arels:
            fout.write(f"{aid}\tQ0\t{pid}\t{1}\n")
            
# for queries
fn_to_qids = {
    "queries.test.tsv": list(test_qid_to_pids.keys()),
    "queries.test.small.tsv": random.sample(test_qid_to_pids.keys(), k=10_000)
}
for fn, qids in fn_to_qids.items():
    with open(os.path.join(out_dir, fn), "w") as fout:
        for qid in qids:
            text = eid_to_text[qid]
            fout.write(f"{qid}\t{text}\t{REL_RELATION}\n")
fn_to_qrels = {
    "qrels.test.tsv": [(qid, pid) for (qid, pids) in test_qid_to_pids.items() for pid in pids],
}
for fn, qrels in fn_to_qrels.items():
    with open(os.path.join(out_dir, fn), "w") as fout:
        for (qid, pid) in qrels:
            fout.write(f"{qid}\tQ0\t{pid}\t{1}\n")

In [20]:
# sanity check
out_dir = os.path.join(kgc_dir, "unified_train/")
for path in os.listdir(out_dir):
    path = os.path.join(out_dir, path)
    ! wc -l $path
    ! head -n 3 $path
    ! tail -n 3 $path
    print("="*100)

600000 /home/jupyter/unity_jointly_rec_and_search/datasets/spotify_mpd/processed/unified_train/q2p.train.tsv
2637595	115507	961223	is_relevant_to
2994756	24198	1070713	is_relevant_to
3080908	42698	1044471	is_relevant_to
2985700	31379	1693708	is_relevant_to
3201060	88837	2159403	is_relevant_to
2944609	3550	400858	is_relevant_to
600000 /home/jupyter/unity_jointly_rec_and_search/datasets/spotify_mpd/processed/unified_train/a2sp.train.tsv
997328	21976	1083169	also_contains
1037363	124411	407725	also_contains
1064371	4148	1086020	also_contains
1072270	13161	615525	also_contains
1208546	502581	1176215	also_contains
1307573	22020	994042	also_contains


In [23]:
hid, pos_tid, neg_tid = (1037363,124411,407725)
print(hid, eid_to_text[hid])
print(pos_tid, eid_to_text[pos_tid])
print(neg_tid, eid_to_text[neg_tid])


1037363 The playlist contains tracks: Track name is Setting Fires in the album Setting Fires by the artist The Chainsmokers ; Track name is Closer in the album Closer by the artist The Chainsmokers ; Track name is Inside Out in the album Inside Out by the artist The Chainsmokers ; Track name is Don't Let Me Down in the album The Chainsmokers- Japan Special Edition by the artist The Chainsmokers ; Track name is Let Me Hold You (Turn Me On) in the album Let Me Hold You (Turn Me On) by the artist Dante Klein ;
124411 Track name is Sapient Dream in the album Brain Freeze by the artist Slushii
407725 Track name is Push - Radio Mix in the album Push by the artist Cannavo & Nesse


In [34]:
# sanity check
import os

#test_dir = os.path.join(in_dir, "selected_test_user")
kgc_dir = "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/"

test_fns = [
    os.path.join(kgc_dir, "unified_test/arels.test.sim.tsv"),
    os.path.join(kgc_dir, "unified_test/arels.test.compl.tsv"),
    os.path.join(kgc_dir, "unified_test/qrels.test.tsv")
]
train_fns = [
    os.path.join(kgc_dir, "unified_train/a2sp.train.tsv"),
    os.path.join(kgc_dir, "unified_train/a2cp.train.tsv"),
    os.path.join(kgc_dir, "unified_train/max2_qorient_q2p.train.tsv")
]

for test_fn, train_fn in zip(test_fns, train_fns):
    test_qids, train_qids = set(), set()
    with open(test_fn) as fin:
        for line in fin:
            array = line.rstrip().split("\t")
            assert len(array) == 4
            test_qids.add(array[0])
    with open(train_fn) as fin:
        for line in fin:
            array = line.rstrip().split("\t")
            assert len(array) == 4
            train_qids.add(array[0])
    assert len(test_qids & train_qids) == 0
    print(f"test_fn: {test_fn}, size: {len(test_qids):,}")
    print(f"train_fn: {train_fn}, size: {len(train_qids):,}")
    print(75*"=")
print("SUCCESS: test and train qids not overlap.")

test_fn: /home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_test/arels.test.sim.tsv, size: 8,273
train_fn: /home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_train/a2sp.train.tsv, size: 199,507
test_fn: /home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_test/arels.test.compl.tsv, size: 6,720
train_fn: /home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_train/a2cp.train.tsv, size: 80,042
test_fn: /home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_test/qrels.test.tsv, size: 7,274
train_fn: /home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_train/max2_qorient_q2p.train.tsv, size: 832,482
SUCCESS: test and train qids not overlap.
